In [1]:
from collections import defaultdict
import tensorflow as tf
import numpy as np
import tflearn
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import tifffile as tiff
import sys
import sklearn
import time
import PIL
from scipy import misc

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def convert_bmp_to_mask(img):
    w, h, c = img.shape
    mask = np.zeros((w, h, c))
    mask[img == 255] = 1
    mask[img < 255] = 0
    mask = np.concatenate([mask, mask[:, :, 0].reshape(w, h, 1)], axis=2)
    return mask

In [3]:
def scale_percentile(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float32)
    # Get 2nd and 98th percentile
    cp_matrix = np.copy(matrix)
    cp_matrix[cp_matrix < 1] = np.nan
    mins = np.nanpercentile(cp_matrix, 1, axis=0)
    maxs = np.nanpercentile(cp_matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

In [4]:
def calculate_std_mean(img):
    w, h, c = img.shape
    target = np.copy(img)
    target[target == 0] = np.nan
    target = target.reshape(-1, c)
    target_std = np.nanstd(target, axis=0)
    target_mean = np.nanmean(target, axis=0)
    return target_std, target_mean

In [5]:
def scale_percentile_2(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float32)
    # Get 2nd and 98th percentile
    zero_pos = matrix < 1
    matrix[zero_pos] = np.nan
    mins = np.nanpercentile(matrix, 1, axis=0)
    maxs = np.nanpercentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix[zero_pos] = 0
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

# Load Boarders and Make Pic Colorful

## 2015

In [9]:
FILE_forward = '/data/zonghua/satellite/Part3/data/quarterfinals_2015.tif'
im_forward = tiff.imread(FILE_forward)
im_forward = np.transpose(im_forward, [1, 2, 0])

In [6]:
forward_1 = cv2.imread("boarders/forward_1.bmp")
forward_2 = cv2.imread("boarders/forward_2.bmp")
forward_3 = cv2.imread("boarders/forward_3.bmp")
forward_4 = cv2.imread("boarders/forward_4.bmp")

In [7]:
where_1 = np.where(forward_1 == 255)
right_1 = min(np.max(where_1[1]) + 100, forward_1.shape[1])
left_1  = max(np.min(where_1[1]) - 100, 0)
forward_1_mask = convert_bmp_to_mask(forward_1[:, left_1: right_1])

In [10]:
mask_forward_part1 = forward_1_mask * im_forward[:, left_1: right_1]
forward_1_target = scale_percentile_2(mask_forward_part1)

ValueError: operands could not be broadcast together with shapes (3000,629,4) (4000,629,4) 

In [ ]:
where_2 = np.where(forward_2 == 255)
right_2 = min(np.max(where_2[1]) + 100, forward_2.shape[1])
left_2  = max(np.min(where_2[1]) - 100, 0)
forward_2_mask = convert_bmp_to_mask(forward_2[:, left_2: right_2])

In [ ]:
mask_forward_part2 = forward_2_mask * im_forward[:, left_2: right_2]
forward_2_target = scale_percentile_2(mask_forward_part2)

In [ ]:
where_3 = np.where(forward_3 == 255)
right_3 = min(np.max(where_3[1]) + 100, forward_3.shape[1])
left_3  = max(np.min(where_3[1]) - 100, 0)
forward_3_mask = convert_bmp_to_mask(forward_3[:, left_3: right_3])

In [ ]:
mask_forward_part3 = forward_3_mask * im_forward[:, left_3: right_3]
forward_3_target = scale_percentile_2(mask_forward_part3)

In [ ]:
where_4 = np.where(forward_4 == 255)
right_4 = min(np.max(where_4[1]) + 100, forward_4.shape[1])
left_4  = max(np.min(where_4[1]) - 100, 0)
forward_4_mask = convert_bmp_to_mask(forward_4[:, left_4: right_4])

In [ ]:
mask_forward_part4 = forward_4_mask * im_forward[:, left_4: right_4]
forward_4_target = scale_percentile_2(mask_forward_part4)

In [ ]:
forward_effective = np.zeros(forward_1.shape[:2] + (4,))
forward_effective[:, left_1: right_1] += forward_1_target
forward_effective[:, left_2: right_2] += forward_2_target
forward_effective[:, left_3: right_3] += forward_3_target
forward_effective[:, left_4: right_4] += forward_4_target

In [19]:
cv2.imwrite('boarders/2015.jpg', forward_effective[:, :, :3] * 255)

True

In [20]:
np.save('/data/zonghua/satellite/Final/new_data/2015_data', forward_effective)